# 视频处理后创建新视频

In [4]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# 检测并绘制车道线
def laneDetect(img) :
    [ysize, xsize, channel] = img.shape             # (368, 640, 3)
    res = np.copy(img)                              # 以防后面要叠加，先复制
    imgGray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)  # 转为灰度图
    thresh = 10                                     # 要求Sobel梯度大于的值

    # 区域顶点
    # 应该区域往下点，就可以让max_line_gap大一点，好连成直线
    left_bottom = [236, 368]
    right_bottom = [351, 368]
    left_top = [286, 260]
    right_top = [325,260]

    # 区域边界直线
    fit_left = np.polyfit((left_bottom[0], left_top[0]), (left_bottom[1], left_top[1]), 1)
    fit_right = np.polyfit((right_bottom[0], right_top[0]), (right_bottom[1], right_top[1]), 1)
    fit_bottom = np.polyfit((left_bottom[0], right_bottom[0]), (left_bottom[1], right_bottom[1]), 1)
    fit_top = np.polyfit((left_top[0], right_top[0]), (left_top[1], right_top[1]), 1)

    # 梯形区域Mask：region_thresholds
    XX, YY = np.meshgrid(np.arange(0, xsize), np.arange(0, ysize))
    region_thresholds = (YY > (XX * fit_left[0] + fit_left[1])) & \
                        (YY > (XX * fit_right[0] + fit_right[1])) & \
                        (YY < (XX * fit_bottom[0] + fit_bottom[1])) & \
                        (YY > (XX * fit_top[0] + fit_top[1]))

    # Sobel算子
    cur = cv2.GaussianBlur(imgGray, (3, 3), 1)
    imgSobel = cv2.Canny(cur, 50, 200)
    imgSobel[~region_thresholds] = 0
    # cv2.imshow("imgSobel", imgSobel)                  # 查看
    # cv2.waitKey(0)

    # Hough变换灰度图：阈值分割+区域选择后的imgSobel
    rho = 1
    theta = np.pi/180
    threshold = 10
    min_line_length = 10
    max_line_gap = 10

    lines = cv2.HoughLinesP(imgSobel, rho, theta, threshold, np.array([]),
                                min_line_length, max_line_gap)

    # 绘制直线：彩色图res
    if lines is not None:
        for line in lines:
            for x1,y1,x2,y2 in line:
                cv2.line(res,(x1, y1), (x2, y2), (0, 255, 0), 2)
    else :
        print("None Lines!")
    
    return res

In [6]:
img = cv2.imread('../pics/img1_10.jpg', 1)              # 这里是彩色图像
res = laneDetect(img)
cv2.imshow("image", res)                  # 查看
cv2.waitKey(0)

-1

In [14]:
# 总共3785张图片，选取40s作为问题一案例
# 视频1的l = 0，视频5的l = 420
l = 420
r = l + 840

file_path = '../data/'
video_name = '5.mp4'
# 读取原视频
videoCapture = cv2.VideoCapture(file_path + video_name)

#获得码率及尺寸
fps = videoCapture.get(cv2.CAP_PROP_FPS)
size = (int(videoCapture.get(cv2.CAP_PROP_FRAME_WIDTH)), 
        int(videoCapture.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fNUMS = videoCapture.get(cv2.CAP_PROP_FRAME_COUNT)

fourcc = cv2.VideoWriter_fourcc(*'MP4V')              # mp4格式保存
# 按照设置的格式来out输出
out = cv2.VideoWriter(file_path + 'out1/out' + video_name,fourcc ,fps, size)

# 读帧处理并保存
success, frame = videoCapture.read()
flag = 0
while success :
    #cv2.imshow('windows', frame)
    # 处理
    if(flag >= l and flag <= r) :
        res = laneDetect(frame)
        
        # 保存
        out.write(res)
    flag = flag + 1
   
    # 获取下一帧
    success, frame = videoCapture.read()

# 释放资源
videoCapture.release()
out.release()

# 关闭窗口
cv2.destroyAllWindows()